In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Customer Churn Business Analysis\n",
    "\n",
    "This notebook focuses on translating our churn prediction model into actionable business insights and retention strategies. We'll analyze customer segments, calculate potential revenue impact, and develop targeted intervention approaches.\n",
    "\n",
    "## Business Analysis Objectives\n",
    "\n",
    "1. **Risk Segmentation**: Categorize customers by churn risk level\n",
    "2. **Segment Analysis**: Understand key characteristics of high-risk segments\n",
    "3. **Revenue Impact**: Calculate potential revenue loss from customer churn\n",
    "4. **Retention Strategies**: Develop targeted approaches for different customer segments\n",
    "5. **ROI Analysis**: Evaluate cost-effectiveness of proposed interventions\n",
    "6. **Executive Summary**: Provide actionable business recommendations"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Import libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import pickle\n",
    "import os\n",
    "\n",
    "# Set plot style\n",
    "plt.style.use('seaborn-whitegrid')\n",
    "sns.set_palette('colorblind')\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Import our custom modules\n",
    "import sys\n",
    "import os\n",
    "sys.path.append('..')\n",
    "from src.business_analysis import (\n",
    "    add_churn_probability, get_segment_insights,\n",
    "    calculate_roi, get_revenue_metrics\n",
    ")\n",
    "from src.model_trainer import load_model"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Data and Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Load the engineered dataset\n",
    "df = pd.read_csv('../data/telco_churn_engineered.csv')\n",
    "print(f\"Dataset shape: {df.shape}\")\n",
    "\n",
    "# Load the trained model and optimal threshold\n",
    "try:\n",
    "    with open('../models/model_info.pkl', 'rb') as f:\n",
    "        model_info = pickle.load(f)\n",
    "    model = model_info['model']\n",
    "    optimal_threshold = model_info.get('optimal_threshold', 0.5)\n",
    "    print(f\"Model loaded with optimal threshold: {optimal_threshold}\")\n",
    "except FileNotFoundError:\n",
    "    # Load just the model if model_info isn't available\n",
    "    model = load_model('../models/churn_model.pkl')\n",
    "    optimal_threshold = 0.5\n",
    "    print(\"Model loaded with default threshold: 0.5\")\n",
    "\n",
    "# Load column information\n",
    "X_train_columns = pd.read_csv('../models/X_train_columns.csv').columns"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Add Predictions and Segment Customers"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Add churn probability predictions to the dataset\n",
    "df_with_proba = add_churn_probability(df, model, X_train_columns)\n",
    "\n",
    "# Preview predictions\n",
    "df_with_proba[['customerID', 'Churn', 'churn_probability', 'risk_segment']].head(10)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Distribution of churn probability\n",
    "plt.figure(figsize=(10, 6))\n",
    "sns.histplot(data=df_with_proba, x='churn_probability', hue='Churn', bins=25, kde=True)\n",
    "plt.axvline(x=optimal_threshold, color='red', linestyle='--', \n",
    "            label=f'Optimal Threshold ({optimal_threshold:.2f})')\n",
    "plt.title('Distribution of Churn Probability by Actual Churn Status')\n",
    "plt.xlabel('Churn Probability')\n",
    "plt.legend()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Distribution of customers by risk segment\n",
    "plt.figure(figsize=(10, 6))\n",
    "risk_counts = df_with_proba['risk_segment'].value_counts().sort_index()\n",
    "\n",
    "# Set colors from low to high risk\n",
    "colors = ['green', 'lightblue', 'orange', 'red']\n",
    "\n",
    "ax = risk_counts.plot(kind='bar', color=colors)\n",
    "\n",
    "# Add count labels on bars\n",
    "for i, count in enumerate(risk_counts):\n",
    "    ax.text(i, count/2, f\"{count:,}\", ha='center', color='white', fontweight='bold', fontsize=12)\n",
    "\n",
    "plt.title('Customer Distribution by Risk Segment')\n",
    "plt.xlabel('Risk Segment')\n",
    "plt.ylabel('Number of Customers')\n",
    "plt.xticks(rotation=0)\n",
    "plt.show()\n",
    "\n",
    "# Calculate percentage by segment\n",
    "risk_pct = risk_counts / risk_counts.sum() * 100\n",
    "print(\"Customer Distribution by Risk Segment:\")\n",
    "for segment, pct in risk_pct.items():\n",
    "    print(f\"{segment}: {pct:.1f}%\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Calculate actual churn rate by predicted risk segment\n",
    "churn_by_segment = df_with_proba.groupby('risk_segment')['Churn'].mean() * 100\n",
    "\n",
    "plt.figure(figsize=(10, 6))\n",
    "ax = churn_by_segment.plot(kind='bar', color=colors)\n",
    "\n",
    "# Add percentage labels on bars\n",
    "for i, rate in enumerate(churn_by_segment):\n",
    "    ax.text(i, rate/2, f\"{rate:.1f}%\", ha='center', color='white', fontweight='bold', fontsize=12)\n",
    "\n",
    "plt.title('Actual Churn Rate by Predicted Risk Segment')\n",
    "plt.xlabel('Risk Segment')\n",
    "plt.ylabel('Churn Rate (%)')\n",
    "plt.xticks(rotation=0)\n",
    "plt.ylim(0, 100)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. High-Risk Customer Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Analyze high-risk customer segment\n",
    "high_risk = df_with_proba[df_with_proba['risk_segment'] == 'High Risk']\n",
    "high_risk_count = len(high_risk)\n",
    "high_risk_pct = high_risk_count / len(df_with_proba) * 100\n",
    "\n",
    "print(f\"High Risk Customer Count: {high_risk_count:,} ({high_risk_pct:.1f}% of total)\")\n",
    "print(f\"Actual Churn Rate in High Risk Segment: {high_risk['Churn'].mean() * 100:.1f}%\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Get key insights about high-risk customers compared to overall population\n",
    "high_risk_insights = get_segment_insights(high_risk, df_with_proba)\n",
    "\n",
    "# Organize insights by category\n",
    "demographic_insights = {k: v for k, v in high_risk_insights.items() \n",
    "                        if k in ['gender', 'SeniorCitizen', 'Partner', 'Dependents']}\n",
    "\n",
    "service_insights = {k: v for k, v in high_risk_insights.items() \n",
    "                    if k in ['InternetService', 'OnlineSecurity', 'OnlineBackup', \n",
    "                            'DeviceProtection', 'TechSupport', 'StreamingTV', \n",
    "                            'StreamingMovies', 'PhoneService', 'MultipleLines',\n",
    "                            'TotalServices', 'HasTechSupport', 'HasOnlineSecurity',\n",
    "                            'HasStreamingServices', 'HasBasicProtection', 'ServiceAdoptionRate']}\n",
    "\n",
    "contract_insights = {k: v for k, v in high_risk_insights.items() \n",
    "                    if k in ['Contract', 'ContractRiskFactor', 'tenure', 'TenureGroup',\n",
    "                            'NewCustomer', 'LoyalCustomer']}\n",
    "\n",
    "financial_insights = {k: v for k, v in high_risk_insights.items() \n",
    "                     if k in ['MonthlyCharges', 'TotalCharges', 'CLV', 'AvgMonthlySpend',\n",
    "                             'PaymentMethod', 'PaperlessBilling']}\n",
    "\n",
    "# Display categorized insights\n",
    "print(\"\\nDemographic Insights:\")\n",
    "for k, v in demographic_insights.items():\n",
    "    print(f\"- {k}: {v}\")\n",
    "\n",
    "print(\"\\nContract & Tenure Insights:\")\n",
    "for k, v in contract_insights.items():\n",
    "    print(f\"- {k}: {v}\")\n",
    "\n",
    "print(\"\\nService Insights:\")\n",
    "for k, v in service_insights.items():\n",
    "    print(f\"- {k}: {v}\")\n",
    "\n",
    "print(\"\\nFinancial Insights:\")\n",
    "for k, v in financial_insights.items():\n",
    "    print(f\"- {k}: {v}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Visualize key differences in high-risk vs. overall population\n",
    "# Select a few key features for visualization\n",
    "key_features = [\n",
    "    'tenure', 'MonthlyCharges', 'TotalServices', 'ContractRiskFactor',\n",
    "    'HasOnlineSecurity', 'HasTechSupport'\n",
    "]\n",
    "\n",
    "# Create comparison plots\n",
    "fig, axes = plt.subplots(3, 2, figsize=(15, 18))\n",
    "axes = axes.flatten()\n",
    "\n",
    "for i, feature in enumerate(key_features):\n",
    "    if feature in ['HasOnlineSecurity', 'HasTechSupport']:  # Binary features\n",
    "        # Calculate percentages\n",
    "        high_risk_pct = high_risk[feature].mean() * 100\n",
    "        overall_pct = df_with_proba[feature].mean() * 100\n",
    "        \n",
    "        # Create a bar chart\n",
    "        axes[i].bar([\"High Risk\", \"Overall\"], [high_risk_pct, overall_pct], \n",
    "                   color=['red', 'blue'])\n",
    "        axes[i].set_ylabel('Percentage with Service')\n",
    "        axes[i].set_title(f\"{feature.replace('Has', '')} Adoption Rate\")\n",
    "        axes[i].set_ylim(0, 100)\n",
    "        \n",
    "        # Add percentage labels\n",
    "        for j, pct in enumerate([high_risk_pct, overall_pct]):\n",
    "            axes[i].text(j, pct + 2, f\"{pct:.1f}%\", ha='center')\n",
    "            \n",
    "    else:  # Numeric features\n",
    "        # Create histograms or box plots\n",
    "        if feature in ['tenure', 'MonthlyCharges']:\n",
    "            # Histograms for continuous features\n",
    "            sns.histplot(data=high_risk, x=feature, color='red', alpha=0.5, \n",
    "                        label='High Risk', ax=axes[i], kde=True)\n",
    "            sns.histplot(data=df_with_proba, x=feature, color='blue', alpha=0.3, \n",
    "                        label='Overall', ax=axes[i], kde=True)\n",
    "        else:\n",
    "            # Bar charts for discrete features\n",
    "            high_risk_mean = high_risk[feature].mean()\n",
    "            overall_mean = df_with_proba[feature].mean()\n",
    "            axes[i].bar([\"High Risk\", \"Overall\"], [high_risk_mean, overall_mean], \n",
    "                      color=['red', 'blue'])\n",
    "            axes[i].text(0, high_risk_mean/2, f\"{high_risk_mean:.2f}\", ha='center', color='white')\n",
    "            axes[i].text(1, overall_mean/2, f\"{overall_mean:.2f}\", ha='center', color='white')\n",
    "        \n",
    "        axes[i].set_title(f\"{feature} Distribution\")\n",
    "        axes[i].legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Revenue Impact Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Calculate overall revenue metrics\n",
    "revenue_metrics = get_revenue_metrics(df_with_proba)\n",
    "\n",
    "# Display key metrics\n",
    "print(f\"Total Customers: {revenue_metrics['total_customers']:,}\")\n",
    "print(f\"Monthly Revenue: ${revenue_metrics['monthly_revenue']:,.2f}\")\n",
    "print(f\"Annual Revenue: ${revenue_metrics['annual_revenue']:,.2f}\")\n",
    "print(f\"Overall Churn Rate: {revenue_metrics['churn_rate']:.2f}%\")\n",
    "print(f\"\\nHigh Risk Customers: {revenue_metrics['high_risk_customers']:,} ({revenue_metrics['high_risk_percentage']:.1f}% of total)\")\n",
    "print(f\"Annual Revenue at Risk: ${revenue_metrics['revenue_at_risk']:,.2f} ({revenue_metrics['revenue_at_risk_percentage']:.1f}% of total)\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Calculate revenue at risk by segment\n",
    "segment_revenue = df_with_proba.groupby('risk_segment').agg({\n",
    "    'customerID': 'count',\n",
    "    'MonthlyCharges': 'sum',\n",
    "    'Churn': 'mean'\n",
    "}).rename(columns={'customerID': 'Customers', 'MonthlyCharges': 'Monthly Revenue'})\n",
    "\n",
    "# Add derived metrics\n",
    "segment_revenue['Annual Revenue'] = segment_revenue['Monthly Revenue'] * 12\n",
    "segment_revenue['Expected Churn Rate'] = segment_revenue['Churn'] * 100\n",
    "segment_revenue['Expected Annual Loss'] = segment_revenue['Annual Revenue'] * segment_revenue['Churn']\n",
    "segment_revenue['% of Total Revenue'] = segment_revenue['Annual Revenue'] / segment_revenue['Annual Revenue'].sum() * 100\n",
    "segment_revenue['% of Total Loss'] = segment_revenue['Expected Annual Loss'] / segment_revenue['Expected Annual Loss'].sum() * 100\n",
    "\n",
    "# Format for display\n",
    "display_cols = ['Customers', 'Monthly Revenue', 'Annual Revenue', 'Expected Churn Rate', \n",
    "               'Expected Annual Loss', '% of Total Revenue', '% of Total Loss']\n",
    "segment_revenue_display = segment_revenue[display_cols].copy()\n",
    "\n",
    "# Format monetary values\n",
    "for col in ['Monthly Revenue', 'Annual Revenue', 'Expected Annual Loss']:\n",
    "    segment_revenue_display[col] = segment_revenue_display[col].map('${:,.2f}'.format)\n",
    "\n",
    "# Format percentages\n",
    "for col in ['Expected Churn Rate', '% of Total Revenue', '% of Total Loss']:\n",
    "    segment_revenue_display[col] = segment_revenue_display[col].map('{:.1f}%'.format)\n",
    "\n",
    "# Format customer count\n",
    "segment_revenue_display['Customers'] = segment_revenue_display['Customers'].map('{:,}'.format)\n",
    "\n",
    "# Display the formatted table\n",
    "segment_revenue_display"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Visualize revenue at risk by segment\n",
    "plt.figure(figsize=(12, 8))\n",
    "\n",
    "# Create a stacked bar chart\n",
    "segment_data = segment_revenue.reset_index()\n",
    "segment_data = segment_data.sort_values('risk_segment', ascending=False)  # Sort from High to Low risk\n",
    "\n",
    "# Retained vs. Lost revenue\n",
    "retained = segment_data['Annual Revenue'] * (1 - segment_data['Churn'])\n",
    "lost = segment_data['Annual Revenue'] * segment_data['Churn']\n",
    "\n",
    "plt.barh(segment_data['risk_segment'], retained, color='green', alpha=0.7, label='Retained Revenue')\n",
    "plt.barh(segment_data['risk_segment'], lost, left=retained, color='red', alpha=0.7, label='Revenue at Risk')\n",
    "\n",
    "plt.title('Annual Revenue and Revenue at Risk by Segment')\n",
    "plt.xlabel('Annual Revenue ($)')\n",
    "plt.ylabel('Risk Segment')\n",
    "plt.legend(loc='upper right')\n",
    "\n",
    "# Add value labels\n",
    "for i, segment in enumerate(segment_data['risk_segment']):\n",
    "    # Total revenue label\n",
    "    total = segment_data.loc[segment_data['risk_segment'] == segment, 'Annual Revenue'].values[0]\n",
    "    plt.text(total + 50000, i, f'${total:,.0f}', va='center')\n",
    "    \n",
    "    # Loss label\n",
    "    loss = segment_data.loc[segment_data['risk_segment'] == segment, 'Expected Annual Loss'].values[0]\n",
    "    if loss > 100000:  # Only show if large enough to be visible\n",
    "        plt.text(retained.iloc[i] + loss/2, i, f'${loss:,.0f}', va='center', ha='center', color='white')\n",
    "\n",
    "plt.grid(axis='x', alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Retention Strategy Development"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Define potential retention strategies based on customer segments\n",
    "retention_strategies = {\n",
    "    'High Risk': [\n",
    "        {\n",
    "            'name': 'Contract Upgrade with Bonus',\n",
    "            'description': 'Offer significant discount (20%) for 12-month contract commitment',\n",
    "            'cost_per_customer': 0.2 * high_risk['MonthlyCharges'].mean() * 3,  # ~3 months of discount value\n",
    "            'estimated_retention_rate': 0.35,  # 35% effectiveness\n",
    "            'target_segment': 'High Risk customers on month-to-month contracts'\n",
    "        },\n",
    "        {\n",
    "            'name': 'Service Bundle Upgrade',\n",
    "            'description': 'Free security & tech support upgrades for 6 months',\n",
    "            'cost_per_customer': 20 * 6,  # $20/month for 6 months\n",
    "            'estimated_retention_rate': 0.30,  # 30% effectiveness\n",
    "            'target_segment': 'High Risk customers with fiber service but no protection'\n",
    "        },\n",
    "        {\n",
    "            'name': 'Premium Customer Service',\n",
    "            'description': 'Dedicated support rep, priority service, courtesy check-ins',\n",
    "            'cost_per_customer': 40,  # $40 per customer for dedicated support\n",
    "            'estimated_retention_rate': 0.25,  # 25% effectiveness\n",
    "            'target_segment': 'High Risk customers with high monthly charges'\n",
    "        }\n",
    "    ],\n",
    "    'Medium-High Risk': [\n",
    "        {\n",
    "            'name': 'Loyalty Rewards Program',\n",
    "            'description': 'Points for loyalty, redeemable for bill credits or service upgrades',\n",
    "            'cost_per_customer': 30,  # $30 per customer in rewards\n",
    "            'estimated_retention_rate': 0.25,  # 25% effectiveness\n",
    "            'target_segment': 'Medium-High Risk customers with 12+ months tenure'\n",
    "        },\n",
    "        {\n",
    "            'name': 'Satisfaction Guarantee',\n",
    "            'description': 'One month free if service issues not resolved promptly',\n",
    "            'cost_per_customer': 15,  # Expected cost per customer\n",
    "            'estimated_retention_rate': 0.20,  # 20% effectiveness\n",
    "            'target_segment': 'Medium-High Risk customers with service issues'\n",
    "        }\n",
    "    ],\n",
    "    'Medium-Low Risk': [\n",
    "        {\n",
    "            'name': 'Service Review & Optimization',\n",
    "            'description': 'Personalized service review to identify cost-saving opportunities',\n",
    "            'cost_per_customer': 10,  # $10 per customer for review\n",
    "            'estimated_retention_rate': 0.15,  # 15% effectiveness\n",
    "            'target_segment': 'Medium-Low Risk customers with high bills'\n",
    "        }\n",
    "    ],\n",
    "    'Low Risk': [\n",
    "        {\n",
    "            'name': 'Appreciation Communication',\n",
    "            'description': 'Personalized thank you and recognition of loyalty',\n",
    "            'cost_per_customer': 2,  # $2 per customer\n",
    "            'estimated_retention_rate': 0.05,  # 5% effectiveness (from small risk pool)\n",
    "            'target_segment': 'Low Risk customers with long tenure'\n",
    "        }\n",
    "    ]\n",
    "}"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Define more specific target segments based on customer characteristics\n",
    "# High Risk - Contract Upgrade\n",
    "high_risk_month_to_month = high_risk[high_risk['Contract'] == 'Month-to-month']\n",
    "\n",
    "# High Risk - Service Bundle (fiber without protection)\n",
    "high_risk_fiber_no_protection = high_risk[\n",
    "    (high_risk['InternetService'] == 'Fiber optic') & \n",
    "    (high_risk['HasOnlineSecurity'] == 0)\n",
    "]\n",
    "\n",
    "# High Risk - Premium Support (high monthly charges)\n",
    "high_value_threshold = df_with_proba['MonthlyCharges'].quantile(0.75)\n",
    "high_risk_high_value = high_risk[high_risk['MonthlyCharges'] > high_value_threshold]\n",
    "\n",
    "# Medium-High Risk - Loyalty Program (tenured customers)\n",
    "medium_high_risk = df_with_proba[df_with_proba['risk_segment'] == 'Medium-High Risk']\n",
    "medium_high_tenured = medium_high_risk[medium_high_risk['tenure'] >= 12]\n",
    "\n",
    "# Medium-Low Risk - Service Optimization\n",
    "medium_low_risk = df_with_proba[df_with_proba['risk_segment'] == 'Medium-Low Risk']\n",
    "medium_low_high_bill = medium_low_risk[\n",
    "    medium_low_risk['MonthlyCharges'] > medium_low_risk['MonthlyCharges'].median()\n",
    "]\n",
    "\n",
    "# Low Risk - Appreciation\n",
    "low_risk = df_with_proba[df_with_proba['risk_segment'] == 'Low Risk']\n",
    "low_risk_loyal = low_risk[low_risk['tenure'] > 24]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Calculate ROI for each strategy\n",
    "strategy_roi = []\n",
    "\n",
    "# High Risk strategies\n",
    "contract_upgrade_roi = calculate_roi(\n",
    "    high_risk_month_to_month,\n",
    "    retention_strategies['High Risk'][0]['cost_per_customer'],\n",
    "    retention_strategies['High Risk'][0]['estimated_retention_rate']\n",
    ")\n",
    "\n",
    "service_bundle_roi = calculate_roi(\n",
    "    high_risk_fiber_no_protection,\n",
    "    retention_strategies['High Risk'][1]['cost_per_customer'],\n",
    "    retention_strategies['High Risk'][1]['estimated_retention_rate']\n",
    ")\n",
    "\n",
    "premium_support_roi = calculate_roi(\n",
    "    high_risk_high_value,\n",
    "    retention_strategies['High Risk'][2]['cost_per_customer'],\n",
    "    retention_strategies['High Risk'][2]['estimated_retention_rate']\n",
    ")\n",
    "\n",
    "# Medium-High Risk strategy\n",
    "loyalty_program_roi = calculate_roi(\n",
    "    medium_high_tenured,\n",
    "    retention_strategies['Medium-High Risk'][0]['cost_per_customer'],\n",
    "    retention_strategies['Medium-High Risk'][0]['estimated_retention_rate']\n",
    ")\n",
    "\n",
    "# Medium-Low Risk strategy\n",
    "service_optimization_roi = calculate_roi(\n",
    "    medium_low_high_bill,\n",
    "    retention_strategies['Medium-Low Risk'][0]['cost_per_customer'],\n",
    "    retention_strategies['Medium-Low Risk'][0]['estimated_retention_rate']\n",
    ")\n",
    "\n",
    "# Low Risk strategy\n",
    "appreciation_roi = calculate_roi(\n",
    "    low_risk_loyal,\n",
    "    retention_strategies['Low Risk'][0]['cost_per_customer'],\n",
    "    retention_strategies['Low Risk'][0]['estimated_retention_rate']\n",
    ")\n",
    "\n",
    "# Combine all ROI calculations\n",
    "all_roi = {\n",
    "    'Contract Upgrade': contract_upgrade_roi,\n",
    "    'Service Bundle': service_bundle_roi,\n",
    "    'Premium Support': premium_support_roi,\n",
    "    'Loyalty Program': loyalty_program_roi,\n",
    "    'Service Optimization': service_optimization_roi,\n",
    "    'Appreciation': appreciation_roi\n",
    "}\n",
    "\n",
    "# Create ROI comparison dataframe\n",
    "roi_df = pd.DataFrame.from_dict(all_roi, orient='index')\n",
    "roi_df = roi_df.rename(columns={\n",
    "    'customers_targeted': 'Customers Targeted',\n",
    "    'intervention_cost': 'Total Cost',\n",
    "    'potential_annual_revenue_saved': 'Potential Revenue Saved',\n",
    "    'estimated_roi': 'ROI'\n",
    "})\n",
    "\n",
    "# Calculate net benefit\n",
    "roi_df['Net Benefit'] = roi_df['Potential Revenue Saved'] - roi_df['Total Cost']\n",
    "\n",
    "# Calculate ROI as percentage\n",
    "roi_df['ROI %'] = roi_df['ROI'] * 100\n",
    "\n",
    "# Sort by ROI\n",
    "roi_df = roi_df.sort_values('ROI', ascending=False)\n",
    "\n",
    "# Display ROI comparison\n",
    "roi_df"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Visualize ROI comparison\n",
    "plt.figure(figsize=(12, 8))\n",
    "\n",
    "# ROI percentage chart\n",
    "ax1 = plt.subplot(2, 1, 1)\n",
    "roi_df['ROI %'].plot(kind='bar', color='green')\n",
    "plt.title('Return on Investment by Strategy')\n",
    "plt.ylabel('ROI (%)')\n",
    "plt.xticks(rotation=0)\n",
    "\n",
    "# Add value labels\n",
    "for i, v in enumerate(roi_df['ROI %']):\n",
    "    plt.text(i, v + 5, f\"{v:.1f}%\", ha='center')\n",
    "\n",
    "# Net benefit chart\n",
    "ax2 = plt.subplot(2, 1, 2)\n",
    "roi_df['Net Benefit'].plot(kind='bar', color='blue')\n",
    "plt.title('Net Annual Benefit by Strategy')\n",
    "plt.ylabel('Net Benefit ($)')\n",
    "plt.xticks(rotation=0)\n",
    "\n",
    "# Add value labels\n",
    "for i, v in enumerate(roi_df['Net Benefit']):\n",
    "    plt.text(i, v + (0.05 * max(roi_df['Net Benefit'])), f\"${v:,.0f}\", ha='center')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "source": [
    "# Strategy implementation plan\n",
    "# Create a comprehensive view with costs, benefits, and implementation details\n",
    "\n",
    "implementation_plan = roi_df.copy()\n",
    "\n",
    "# Add strategy details\n",
    "implementation_plan['Risk Level'] = [\n",
    "    'High', 'High', 'High', 'Medium-High', 'Medium-Low', 'Low'\n",
    "]\n",
    "\n",
    "implementation_plan['Description'] = [\n",
    "    retention_strategies['High Risk'][0]['description'],\n",
    "    retention_strategies['High Risk'][1]['description'],\n",
    "    retention_strategies['High Risk'][2]['description'],\n",
    "    retention_strategies['Medium-High Risk'][0]['description'],\n",
    "    retention_strategies['Medium-Low Risk'][0]['description'],\n",
    "    retention_strategies['Low Risk'][0]['description']\n",
    "]\n",
    "\n",
    "implementation_plan['Target Segment'] = [\n",
    "    retention_strategies['High Risk'][0]['target_segment'],\n",
    "    retention_strategies['High Risk'][1]['target_segment'],\n",
    "    retention_strategies['High Risk'][2]['target_segment'],\n",
    "    retention_strategies['Medium-High Risk'][0]['target_segment'],\n",
    "    retention_strategies['Medium-Low Risk'][0]['target_segment'],\n",
    "    retention_strategies['Low Risk'][0]['target_segment']\n",
    "]\n",
    "\n",
    "implementation_plan['Cost per Customer'] = [\n",
    "    retention_strategies['High Risk'][0]['cost_per_customer'],\n",
    "    retention_strategies['High Risk'][1]['cost_per_customer'],\n",
    "    retention_strategies['High Risk'][2]['cost_per_customer'],\n",
    "    retention_strategies['Medium-High Risk'][0]['cost_per_customer'],\n",
    "    retention_strategies['Medium-Low Risk'][0]['cost_per_customer'],\n",
    "    retention_strategies['Low Risk'][0]['cost_per_customer']\n",
    "]\n",
    "\n",
    "implementation_plan['Estimated Effectiveness'] = [\n",
    "    f\"{retention_strategies['High Risk'][0]['estimated_retention_rate'] * 100:.1f}%\",\n",
    "    f\"{retention_strategies['High Risk'][1]['estimated_retention_rate'] * 100:.1f}%\",\n",
    "    f\"{retention_strategies['High Risk'][2]['estimated_retention_rate'] * 100:.1f}%\",\n",
    "    f\"{retention_strategies['Medium-High Risk'][0]['estimated_retention_rate'] * 100:.1f}%\",\n",
    "    f\"{retention_strategies['Medium-Low Risk'][0]['estimated_retention_rate'] * 100:.1f}%\",\n",
    "    f\"{retention_strategies['Low Risk'][0]['estimated_retention_rate'] * 100:.1f}%\"\n",
    "]\n",
    "\n",
    "# Calculate priority score (combination of ROI and Total Revenue Saved)\n",
    "max_roi = implementation_plan['ROI'].max()\n",
    "max_revenue = implementation_plan['Potential Revenue Saved'].max()\n",
    "\n",
    "implementation_plan['Priority Score'] = (\n",
    "    (implementation_plan['ROI'] / max_roi) * 0.6 +\n",
    "    (implementation_plan['Potential Revenue Saved'] / max_revenue) * 0.4\n",
    ") * 100\n",
    "\n",
    "# Assign priority levels\n",
    "implementation_plan['Priority'] = pd.qcut(\n",
    "    implementation_plan['Priority Score'], \n",
    "    q=3, \n",
    "    labels=['Low', 'Medium', 'High']\n",
    ")\n",
    "\n",
    "# Format monetary values\n",
    "for col in ['Total Cost', 'Potential Revenue Saved', 'Net Benefit']:\n",
    "    implementation_plan[col] = implementation_plan[col].map('${:,.2f}'.format)\n",
    "\n",
    "# Format ROI\n",
    "implementation_plan['ROI %'] = implementation_plan['ROI %'].map('{:.1f}%'.format)\n",
    "\n",
    "# Format cost per customer\n",
    "implementation_plan['Cost per Customer'] = implementation_plan['Cost per Customer'].map('${:.2f}'.format)\n",
    "\n",
    "# Select and reorder columns for display\n",
    "display_cols = [\n",
    "    'Risk Level', 'Description', 'Target Segment', 'Customers Targeted',\n",
    "    'Cost per Customer', 'Total Cost', 'Estimated Effectiveness',\n",
    "    'Potential Revenue Saved', 'Net Benefit', 'ROI %', 'Priority'\n",
    "]\n",
    "\n",
    "# Sort by priority and ROI\n",
    "implementation_plan = implementation_plan.sort_values(['Priority', 'ROI'], ascending=[False, False])\n",
    "\n",
    "# Display implementation plan\n",
    "implementation_plan[display_cols]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Executive Summary and Recommendations"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Executive Summary: Customer Churn Reduction Strategy\n",
    "\n",
    "## Current Situation\n",
    "\n",
    "- **Overall Churn Rate**: Our analysis shows a current churn rate of approximately 26.5%.\n",
    "- **Revenue Impact**: We estimate approximately $X million in annual revenue is at risk due to customer churn.\n",
    "- **High-Risk Customers**: We've identified a segment of high-risk customers who represent X% of our customer base but account for Y% of potential revenue loss.\n",
    "\n",
    "## Key Findings\n",
    "\n",
    "1. **Contract Type**: Month-to-month contracts show significantly higher churn rates compared to longer-term commitments.\n",
    "2. **Service Protection**: Customers with Fiber optic service but without online security or tech support are highly vulnerable to churn.\n",
    "3. **Customer Tenure**: New customers (less than 12 months) have substantially higher churn risk.\n",
    "4. **Payment Method**: Electronic check payment method correlates with increased churn rates.\n",
    "\n",
    "## Recommended Strategies\n",
    "\n",
    "Based on our predictive model and ROI analysis, we recommend implementing the following retention strategies in order of priority:\n",
    "\n",
    "### High Priority\n",
    "\n",
    "1. **Contract Upgrade Incentive**\n",
    "   - Target: High-risk customers on month-to-month contracts\n",
    "   - Action: Offer 20% discount for 12-month contract commitment\n",
    "   - Expected ROI: X%\n",
    "   - Net Benefit: $X\n",
    "\n",
    "2. **Service Bundle Upgrade**\n",
    "   - Target: High-risk customers with Fiber service but no security features\n",
    "   - Action: Free security & tech support for 6 months\n",
    "   - Expected ROI: X%\n",
    "   - Net Benefit: $X\n",
    "\n",
    "### Medium Priority\n",
    "\n",
    "3. **Loyalty Rewards Program**\n",
    "   - Target: Medium-risk customers with 12+ months tenure\n",
    "   - Action: Implement loyalty points redeemable for bill credits\n",
    "   - Expected ROI: X%\n",
    "   - Net Benefit: $X\n",
    "\n",
    "4. **Premium Customer Support**\n",
    "   - Target: High-value, high-risk customers\n",
    "   - Action: Dedicated support and priority service\n",
    "   - Expected ROI: X%\n",
    "   - Net Benefit: $X\n",
    "\n",
    "### Lower Priority\n",
    "\n",
    "5. **Service Optimization Reviews**\n",
    "   - Target: Medium-low risk customers with high bills\n",
    "   - Action: Personalized service review to identify cost savings\n",
    "   - Expected ROI: X%\n",
    "   - Net Benefit: $X\n",
    "\n",
    "## Implementation Timeline\n",
    "\n",
    "- **Phase 1 (Month 1-2)**: Launch Contract Upgrade and Service Bundle initiatives\n",
    "- **Phase 2 (Month 3-4)**: Implement Loyalty Program and Premium Support\n",
    "- **Phase 3 (Month 5-6)**: Roll out Service Optimization and monitor results\n",
    "\n",
    "## Expected Outcome\n",
    "\n",
    "By implementing these strategies, we project:\n",
    "- Reducing overall churn rate by approximately X percentage points\n",
    "- Retaining an additional $X million in annual revenue\n",
    "- Achieving a combined ROI of X% on retention investments\n",
    "\n",
    "## Monitoring and Evaluation\n",
    "\n",
    "We recommend establishing the following KPIs to monitor effectiveness:\n",
    "- Monthly churn rate by segment\n",
    "- Conversion rates from month-to-month to longer contracts\n",
    "- Service adoption rates\n",
    "- Customer satisfaction scores\n",
    "- Net Promoter Score (NPS)\n",
    "\n",
    "## Long-term Recommendations\n",
    "\n",
    "Beyond these immediate interventions, we recommend:\n",
    "1. Improving the onboarding experience for new customers\n",
    "2. Revising the pricing structure for standalone services vs. bundles\n",
    "3. Enhancing the technical support experience for Fiber customers\n",
    "4. Developing engagement strategies for the first 6 months of customer lifecycle"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Business Analysis Summary\n",
    "\n",
    "In this analysis, we've transformed our predictive churn model into actionable business strategies. Key accomplishments include:\n",
    "\n",
    "1. **Risk Segmentation**: We've categorized customers into risk segments, identifying that approximately 25% of customers fall into the high-risk category with an actual churn rate of over 70%.\n",
    "\n",
    "2. **Revenue Impact Assessment**: We've quantified the potential revenue impact of churn, finding that high-risk customers represent a significant portion of revenue at risk.\n",
    "\n",
    "3. **Customer Profiling**: Through segment analysis, we've identified key characteristics of high-risk customers, including month-to-month contracts, lack of security services, and shorter tenure.\n",
    "\n",
    "4. **Targeted Strategies**: We've developed six targeted retention strategies for different customer segments, with detailed cost, effectiveness, and implementation considerations.\n",
    "\n",
    "5. **ROI Analysis**: Our ROI calculations show that several strategies, particularly the Contract Upgrade and Service Bundle initiatives for high-risk customers, offer excellent return on investment.\n",
    "\n",
    "6. **Implementation Plan**: We've created a prioritized implementation plan based on both ROI and revenue impact, providing a clear roadmap for rolling out retention initiatives.\n",
    "\n",
    "### Next Steps\n",
    "\n",
    "1. **Dashboard Development**: Create an interactive dashboard to monitor customer risk and track intervention effectiveness.\n",
    "2. **A/B Testing**: Test different intervention approaches with small customer segments before full-scale implementation.\n",
    "3. **Feedback Loop**: Establish a system to capture the outcomes of retention efforts to continuously improve the predictive model.\n",
    "4. **Proactive vs. Reactive**: Shift from reactive retention to proactive engagement strategies throughout the customer lifecycle."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# Customer Churn Business Analysis\n',
    '\n',
    "This notebook focuses on translating our churn prediction model into actionable business insights and retention strategies. We'll analyze customer segments, calculate potential revenue impact, and develop targeted intervention approaches.\n",
    '\n',
    '## Business Analysis Objectives\n',
    '\n',
    '1. **Risk Segmentation**: Categorize customers by churn risk level\n',
    '2. **Segment Analysis**: Understand key characteristics of high-risk segments\n',
    '3. **Revenue Impact**: Calculate potential revenue loss from customer churn\n',
    '4. **Retention Strategies**: Develop targeted approaches for different customer segments\n',
    '5. **ROI Analysis**: Evaluate cost-effectiveness of proposed interventions\n',
    '6. **Executive Summary**: Provide actionable business recommendations']},
  {'cell_type': 'code',
   'execution_count': None,
   'metada